In [8]:
import os
import numpy as np
import cv2
from PIL import Image
from operator import itemgetter
import tools as tl
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from matplotlib import pylab as plt
from scipy.stats import entropy, norm
import gc

In [9]:
data='ips'
method='unet_patch'
model_name="unet"

In [10]:
#Directry を作る関数
def my_makedirs(path):
    if not os.path.isdir(path):
        os.makedirs(path)

In [11]:
#正規分布に基づいたstep幅を決める関数
sigma=0.4#0~1の間, 0.4がbest
def gaussian_dist(sigma,max_step):
    X = np.arange(0,1,0.01)
    Y = norm.pdf(X,0,sigma)
    h = 1/Y[0]*max_step
    Y = Y*h
    return Y

In [12]:
def pre_data_path_load(path):#一つ前のCurriculumでのuncertainty mapをloadする関数
    img_path=[]
    if data in ['ips', 'melanoma']:
        for x in os.listdir(path):
            img_path.append(path+x)
        return img_path

In [13]:
#label付けする関数. ipsなら赤が1, 緑が2, 青が3のmapにする. melanomaは0か1
#mask_array...mask画像
#data..."ips"or"melanoma"
def get_ytrue(mask_array, data):
        if data == "ips":
            height, width, _ = mask_array.shape
            good_label = ((mask_array[:,:,0] == 255)&
                          (mask_array[:,:,1] == 0)&
                          (mask_array[:,:,2] == 0)
                         ) * np.ones((height, width)) * 1
            bad_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 255)&
                         (mask_array[:,:,2] == 0)
                        ) * np.ones((height, width)) * 2
            bgd_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 0)&
                         (mask_array[:,:,2] == 255)
                        ) * np.ones((height, width)) * 3
            y_true = good_label + bad_label + bgd_label
            return y_true
        else:
            height, width = mask_array.shape
            y_true = (mask_array == 255) * np.ones((height, width))
            return y_true

In [15]:
#等間隔でpatchを切り出す
number=0
step=40
patch_size=160
for w in ['train','val']:#'train','test'
    for data_num in [1]:
        m=0
        print(w)
        img_list, mask_list = tl.data_path_load(data,w,data_num)
        for i_path, m_path in zip(img_list, mask_list): 
            my_makedirs('/home/sora/new_project/crop/dataset_%d/%s/'%(data_num,w))#このdirにnumpyを保存
            y=0
            print(number)
            number+=1
            
            m_mask = cv2.imread(m_path)#ipsマスク画像の読み込み. 
            m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
            imim = m_mask[:,:,0]+m_mask[:,:,1]+m_mask[:,:,2]
            m_mask[:,:,0]=(imim==255)*m_mask[:,:,0] #紫の部分を0(黒)にしている
            m_mask[:,:,1]=(imim==255)*m_mask[:,:,1]
            m_mask[:,:,2]=(imim==255)*m_mask[:,:,2]
            height, width = m_mask.shape[:2]

            img = Image.open(i_path)#コロニー画像の読み込み
            img = np.array(img)

            while y < (height - patch_size + 1):
                x = 0
                while x < (width - patch_size + 1):
                    list=[]
                    crop_mask = m_mask[y:patch_size+y, x:patch_size+x]#patchの切り出し

                    crop_img = img[y:patch_size+y, x:patch_size+x]
                    c = get_ytrue(crop_mask, 'ips')#label mapを作成
                    if np.count_nonzero(c==0)<patch_size*patch_size*0.2:#label mapを見て, 紫の部分(0)が80%以上ある場合はデータを保存しない
                        crop_mask = crop_mask/255#正規化
                        crop_img = crop_img/255.

                        list.append(crop_img)#listにsetで保存
                        list.append(crop_mask)
                        np.save('/home/sora/new_project/crop/dataset_%d/%s/input%d.npy' %(data_num,w,m), np.array(list))
                        m+=1
                    x+=step   
                y+=step

train
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
val
35
36
37
38
39
40
41
42
43
44
45
46


In [8]:
#patchを切り出す。正規分布に従って横のstep幅を決定する。縦は10pixel。
number=0
step=10
dist = gaussian_dist(sigma, patch_size)#s正規分布作成
for data_num in [1,2,3,4,5]:#データセットの番号(１...5)
    for w in ['train']:#'train','test'
        path=("./%s/%s/%s/size_%d/dataset_%d/%s_un/uncertainty_T_0.5/" %(data, method,model_name,patch_size,data_num,w))
        m=0
        print(w)
        img_list, mask_list = tl.data_path_load(data,w,data_num)#画像のpathを取得
        pre_list = pre_data_path_load(path)#前段階のcurriculu学習のuncertain mapのpathを取得
        img_list.sort()
        mask_list.sort()
        pre_list.sort()
        for i_path, m_path,p_path in zip(img_list, mask_list, pre_list): 
            my_makedirs('/home/sora/new_project/crop/dataset_%d/%s/'%(data_num,w))
            y=0
            print(number)
            number+=1
            m_mask = cv2.imread(m_path)
            m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
            imim = m_mask[:,:,0]+m_mask[:,:,1]+m_mask[:,:,2]
            m_mask[:,:,0]=(imim==255)*m_mask[:,:,0]
            m_mask[:,:,1]=(imim==255)*m_mask[:,:,1]
            m_mask[:,:,2]=(imim==255)*m_mask[:,:,2]
            height, width = m_mask.shape[:2]
            
            mask = get_ytrue(m_mask, 'ips')
            oor = ~(mask == 0) *1#紫の部分を1,他を0とした２次元配列

            img = Image.open(i_path)
            img = np.array(img)


            p_image = cv2.imread(p_path)
            p_image = cv2.cvtColor(p_image, cv2.COLOR_BGR2RGB)
            p_image[:,:,0] = p_image[:,:,0]* oor#uncertain mapの紫の部分を無視するようにしている
            p_image[:,:,1] = p_image[:,:,1] * oor 
            p_image[:,:,2] = p_image[:,:,2] * oor

            while y < (height - patch_size + 1):
                x = 0
                while x < (width - patch_size + 1):
                    list=[]
                    crop_mask = m_mask[y:patch_size+y, x:patch_size+x]

                    crop_img = img[y:patch_size+y, x:patch_size+x]  
                    crop_p = p_image[y:patch_size+y, x:patch_size+x]

                    c = get_ytrue(crop_mask, 'ips')
                    if np.count_nonzero(c==0)<patch_size*patch_size*0.2:               
                        crop_mask = crop_mask/255
                        crop_img = crop_img/255.
                        list.append(crop_img)
                        list.append(crop_mask)

                        np.save('/home/sora/new_project/crop/dataset_%d/%s/input%d.npy' %(data_num,w,m), np.array(list))
                        m+=1
                    e = np.count_nonzero(crop_p[:,:,0] == 255)/(patch_size*patch_size)#uncertain mapの白い部分の割合を計算
                    e = round(e, 3)
                    if e == 1:
                        e = 0.999
                    x_step = int(dist[int(e*100)])#正規分布にて横のstep幅の決定
                    x+=x_step
                y+=step

train
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
train
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
train
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
train
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
train
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
